# Annotated Browsers for Journal-Specific Quotations in *Middlemarch*

In [1]:
import pandas as pd
import nltk
%matplotlib inline
import math
from ast import literal_eval
import numpy as np
import re
from matplotlib import pyplot as plt
#from colour import Color
from IPython.core.display import HTML
from matplotlib import cm
from matplotlib.colors import rgb2hex
plt.rcParams["figure.figsize"] = [16, 6]

In [2]:
df = pd.read_json('../data/t2-c3-n2-m3-no-stops.json')

In [3]:
df

,creator,datePublished,docSubType,docType,id,identifier,isPartOf,issueNumber,language,outputFormat,...,title,url,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,abstract,keyphrase,subTitle
0,[Rainer Emig],2006-01-01,book-review,article,http://www.jstor.org/stable/41158244,"[{'name': 'issn', 'value': '03402827'}, {'name...",Amerikastudien / American Studies,3,[eng],"[unigram, bigram, trigram]",...,Review Article,http://www.jstor.org/stable/41158244,51,1109,1,"[[130022, 130046]]","[[6851, 6875]]",None,None,None
1,[Martin Green],1970-01-01,book-review,article,http://www.jstor.org/stable/3722819,"[{'name': 'issn', 'value': '00267937'}, {'name...",The Modern Language Review,1,[eng],"[unigram, bigram, trigram]",...,Review Article,http://www.jstor.org/stable/3722819,65,1342,0,[],[],None,None,None
2,[Richard Exner],1982-01-01,book-review,article,http://www.jstor.org/stable/40137021,"[{'name': 'issn', 'value': '01963570'}, {'name...",World Literature Today,1,[eng],"[unigram, bigram, trigram]",...,Review Article,http://www.jstor.org/stable/40137021,56,493,0,[],[],None,None,None
3,[Ruth Evelyn Henderson],1925-10-01,research-article,article,http://www.jstor.org/stable/802346,"[{'name': 'issn', 'value': '00138274'}, {'name...",The English Journal,8,[eng],"[unigram, bigram, trigram, fullText]",...,American Education Week--November 16-22; Some ...,http://www.jstor.org/stable/802346,14,2161,0,[],[],None,None,None
4,[Alan Palmer],2011-12-01,research-article,article,http://www.jstor.org/stable/10.5325/style.45.4...,"[{'name': 'issn', 'value': '00394238'}, {'name...",Style,4,[eng],"[unigram, bigram, trigram]",...,Rejoinder to Response by Marie-Laure Ryan,http://www.jstor.org/stable/10.5325/style.45.4...,45,1127,0,[],[],None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5879,[Michaela Giesenkirchen],2005-10-01,research-article,article,http://www.jstor.org/stable/27747183,"[{'name': 'issn', 'value': '15403084'}, {'name...",American Literary Realism,1,[eng],"[unigram, bigram, trigram]",...,Ethnic Types and Problems of Characterization ...,http://www.jstor.org/stable/27747183,38,7349,1,"[[23799, 24121]]","[[41472, 41793]]",None,None,None
5880,[Leon Botstein],2005-07-01,misc,article,http://www.jstor.org/stable/4123220,"[{'name': 'issn', 'value': '00274631'}, {'name...",The Musical Quarterly,2,[eng],"[unigram, bigram, trigram]",...,On the Power of Music,http://www.jstor.org/stable/4123220,88,1525,0,[],[],None,None,None
5881,[Linda M. Shires],2013-01-01,research-article,article,http://www.jstor.org/stable/24575734,"[{'name': 'issn', 'value': '10601503'}, {'name...",Victorian Literature and Culture,4,[eng],"[unigram, bigram, trigram]",...,"HARDY'S MEMORIAL ART: IMAGE AND TEXT IN ""WESSE...",http://www.jstor.org/stable/24575734,41,10736,1,"[[173657, 173756]]","[[33963, 34061]]",None,None,None
5882,[Edward H. Cohen],1990-07-01,misc,article,http://www.jstor.org/stable/3827815,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,Victorian Bibliography for 1989,http://www.jstor.org/stable/3827815,33,81819,0,[],[],None,None,None


In [4]:
df['year'] = pd.DatetimeIndex(df['datePublished']).year

In [5]:
df['Decade'] = df['year'] - (df['year'] % 10)

In [6]:
df['isPartOf'].value_counts()

Victorian Studies                            459
George Eliot - George Henry Lewes Studies    231
The Modern Language Review                   192
Nineteenth-Century Fiction                   192
The Review of English Studies                190
                                            ... 
KulturPoetik                                   1
African Affairs                                1
Western American Literature                    1
The Great Lakes Review                         1
The Musical Quarterly                          1
Name: isPartOf, Length: 789, dtype: int64

In [10]:
victorianist_journals = ['Victorian Studies', 'George Eliot - George Henry Lewes Studies', 'Nineteenth-Century Fiction', 'Nineteenth-Century Literature', 'Dickens Studies Annual', 'Victorian Literature and Culture', 'Victorian Review', 'The George Eliot, George Henry Lewes Newsletter', 'Victorian Periodicals Review', 'Dickens Quarterly', 'Victorian Poetry', 'The Thomas Hardy Journal', 'The Gaskell Society Journal', 'The Gaskell Journal', 'Newsletter of the Victorian Studies Association of Western Canada', 'Dickens Studies Newsletter', 'Browning Institute Studies', 'Victorian Periodicals Newsletter', 'Carlyle Studies Annual', 'Conradiana', 'Tennyson Research Bulletin', 'The Conradian', 'The Hardy Society Journal', 'The Hardy Review', 'Studies in Browning and His Circle', 'Nineteenth-Century French Studies', 'The Wilkie Collins Journal', 'Carlyle Newsletter', 'The Wildean', 'Dickens Studies', 'Carlyle Annual', '19th-Century Music', 'The Trollopian', 'Conrad Studies']

In [20]:
tfile = open('../../victorianist_journals_titles_all.txt', 'a')
tfile.write(str(victorianist_journals))
tfile.close()

## All Victorianist journals

In [15]:
victorianist_df = df[df['isPartOf'].isin(victorianist_journals)]
victorianist_df

,creator,datePublished,docSubType,docType,id,identifier,isPartOf,issueNumber,language,outputFormat,...,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,abstract,keyphrase,subTitle,year,Decade
5,None,2013-01-01,misc,article,http://www.jstor.org/stable/44371392,"[{'name': 'issn', 'value': '00849812'}, {'name...",Dickens Studies Annual,None,[eng],"[unigram, bigram, trigram]",...,44,2235,0,[],[],None,None,None,2013,2010
6,[Mary H Kuhlman],1996-01-01,research-article,article,http://www.jstor.org/stable/45185621,"[{'name': 'issn', 'value': '09517200'}, {'name...",The Gaskell Society Journal,None,[eng],"[unigram, bigram, trigram]",...,10,5672,0,[],[],None,None,None,1996,1990
10,[Cheryl Cassidy],1992-12-01,misc,article,http://www.jstor.org/stable/20082630,"[{'name': 'issn', 'value': '07094698'}, {'name...",Victorian Periodicals Review,4,[eng],"[unigram, bigram, trigram]",...,25,15737,0,[],[],None,None,None,1992,1990
14,[Catherine Gallagher],2006-10-01,book-review,article,http://www.jstor.org/stable/4618956,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,1,[eng],"[unigram, bigram, trigram]",...,49,1378,0,[],[],None,None,None,2006,2000
16,[Angelique Richardson],2006-07-01,book-review,article,http://www.jstor.org/stable/4618943,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,48,1482,0,[],[],None,None,None,2006,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5875,None,2011-10-01,misc,article,http://www.jstor.org/stable/10.2979/victorians...,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,1,[eng],"[unigram, bigram, trigram]",...,54,2413,0,[],[],None,None,None,2011,2010
5876,[BUFF LINDAU],2013-10-01,research-article,article,http://www.jstor.org/stable/42827928,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,64/65,[eng],"[unigram, bigram, trigram]",...,None,129,0,[],[],None,None,None,2013,2010
5881,[Linda M. Shires],2013-01-01,research-article,article,http://www.jstor.org/stable/24575734,"[{'name': 'issn', 'value': '10601503'}, {'name...",Victorian Literature and Culture,4,[eng],"[unigram, bigram, trigram]",...,41,10736,1,"[[173657, 173756]]","[[33963, 34061]]",None,None,None,2013,2010
5882,[Edward H. Cohen],1990-07-01,misc,article,http://www.jstor.org/stable/3827815,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,33,81819,0,[],[],None,None,None,1990,1990


In [22]:
victorianist_df.to_json('../data/victorianist_journals.json')

## *Victorian Studies*

In [23]:
victorian_studies_df = df[df['isPartOf'] == 'Victorian Studies']
victorian_studies_df

,creator,datePublished,docSubType,docType,id,identifier,isPartOf,issueNumber,language,outputFormat,...,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,abstract,keyphrase,subTitle,year,Decade
14,[Catherine Gallagher],2006-10-01,book-review,article,http://www.jstor.org/stable/4618956,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,1,[eng],"[unigram, bigram, trigram]",...,49,1378,0,[],[],None,None,None,2006,2000
16,[Angelique Richardson],2006-07-01,book-review,article,http://www.jstor.org/stable/4618943,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,48,1482,0,[],[],None,None,None,2006,2000
31,None,1960-03-01,misc,article,http://www.jstor.org/stable/3825496,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,3,[eng],"[unigram, bigram, trigram]",...,3,1043,0,[],[],None,None,None,1960,1960
39,[A. S. Crehan],1976-03-01,research-article,article,http://www.jstor.org/stable/3826133,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,3,[eng],"[unigram, bigram, trigram]",...,19,13133,0,[],[],None,None,None,1976,1970
41,[Ronald E. Freeman],1968-06-01,misc,article,http://www.jstor.org/stable/3825239,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,11,36967,0,[],[],None,None,None,1968,1960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,[Chris R. Vanden Bossche],1987-07-01,book-review,article,http://www.jstor.org/stable/3828188,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,30,1676,0,[],[],None,None,None,1987,1980
5873,[Anita Levy],1996-04-01,book-review,article,http://www.jstor.org/stable/3829465,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,3,[eng],"[unigram, bigram, trigram]",...,39,1588,0,[],[],None,None,None,1996,1990
5874,None,1990-07-01,misc,article,http://www.jstor.org/stable/3827789,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,33,4595,0,[],[],None,None,None,1990,1990
5875,None,2011-10-01,misc,article,http://www.jstor.org/stable/10.2979/victorians...,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,1,[eng],"[unigram, bigram, trigram]",...,54,2413,0,[],[],None,None,None,2011,2010


In [24]:
victorian_studies_df.to_json('../data/victorian_studies.json')

## *George Eliot - George Henry Lewes Studies* (*GE-GHLS*)

In [13]:
ge_ghls_df = df[df['isPartOf'] == 'George Eliot - George Henry Lewes Studies']
ge_ghls_df

,creator,datePublished,docSubType,docType,id,identifier,isPartOf,issueNumber,language,outputFormat,...,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,abstract,keyphrase,subTitle,year,Decade
37,[ELIZABETH WINSTON],1995-09-01,book-review,article,http://www.jstor.org/stable/43595523,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,28/29,[eng],"[unigram, bigram, trigram]",...,None,1981,0,[],[],None,None,None,1995,1990
76,[Katherine Newey],2011-09-01,research-article,article,http://www.jstor.org/stable/42827892,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,60/61,[eng],"[unigram, bigram, trigram]",...,None,7038,1,"[[502448, 502471]]","[[18540, 18563]]",None,None,None,2011,2010
101,None,2015-11-01,other,article,http://www.jstor.org/stable/10.5325/georeliogh...,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,2,[eng],"[unigram, bigram, trigram]",...,67,438,0,[],[],None,None,None,2015,2010
107,[AVROM FLEISHMAN],2008-09-01,research-article,article,http://www.jstor.org/stable/42827960,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,54/55,[eng],"[unigram, bigram, trigram]",...,None,22729,1,"[[1138948, 1138968]]","[[73073, 73093]]",None,None,None,2008,2000
108,[Judith Adler],2018-10-01,research-article,article,http://www.jstor.org/stable/10.5325/georeliogh...,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,2,[eng],"[unigram, bigram, trigram]",...,70,9258,1,"[[1792915, 1793447]]","[[350, 876]]",This article argues that the famous concluding...,None,None,2018,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5798,[TERENCE R. WRIGHT],1995-09-01,book-review,article,http://www.jstor.org/stable/43595524,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,28/29,[eng],"[unigram, bigram, trigram]",...,None,862,0,[],[],None,None,None,1995,1990
5835,[SALEEL NURBHAI],1997-09-01,research-article,article,http://www.jstor.org/stable/42827636,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,32/33,[eng],"[unigram, bigram, trigram]",...,None,6755,5,"[[190333, 190518], [939772, 940069], [940403, ...","[[30280, 30465], [30822, 31114], [31125, 31327...",None,None,None,1997,1990
5853,[DONALD HAWES],2001-09-01,research-article,article,http://www.jstor.org/stable/42827734,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,40/41,[eng],"[unigram, bigram, trigram]",...,None,2902,1,"[[1316376, 1316406]]","[[3676, 3706]]",None,None,None,2001,2000
5865,None,1995-09-01,misc,article,http://www.jstor.org/stable/43595525,"[{'name': 'issn', 'value': '23721901'}, {'name...",George Eliot - George Henry Lewes Studies,28/29,[eng],"[unigram, bigram, trigram]",...,None,1147,0,[],[],None,None,None,1995,1990


In [6]:
ge_ghls_df.to_json('../data/ge_ghls_data.json')

In [7]:
ge_ghls_1990s_df = ge_ghls_df[ge_ghls_df['Decade'] == 1990]
ge_ghls_1990s_df.head()

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
14,"[[195357, 195597]]","[[27642, 27885]]","[ROSEMARY BRENNAN DAY, GRAHAM HANDLEY, DONALD ...",[19940901],"[Language & Literature, Humanities]",10.2307/42870840,72ab47e3-5bd7-3c8c-a03e-ee9281130e2d,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[MIDDLEMARCH ON TV--A SYMPOSIUM],"[Villains, Humor, Screenwriting, Pity, Soap op...",fla,None,1994,1990,45,"[[[195357, 195597], 45]]",[45]
34,"[[512101, 512375]]","[[9829, 10102]]",[LESLEY GORDON],[19950901],"[Language & Literature, Humanities]",10.2307/43595510,c7b950b2-0669-3b7e-b270-1f5935d046d2,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[GEORGE ELIOT AND PLUTARCH],"[Nightingales, Tyranny, Theater criticism, Nov...",fla,None,1995,1990,58,"[[[512101, 512375], 58]]",[58]
46,"[[79888, 79968], [137185, 137297], [415296, 41...","[[2131, 2207], [3016, 3127], [7459, 7622], [77...",[CHESTER ST. H. MILLS],[19940901],"[Language & Literature, Humanities]",10.2307/42870836,bc692f3b-e2eb-3f26-9285-9203f5d6de48,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,"[ELIOT'S SPANISH CONNECTION: CASAUBON, THE AVA...","[Chivalry, Somatosensory perception, Endangere...",fla,None,1994,1990,281,"[[[79888, 79968], 19], [[137185, 137297], 30],...","[19, 30, 31, 97, 104]"
49,"[[234173, 234366], [281678, 281814], [539647, ...","[[9325, 9519], [13304, 13440], [13997, 14107],...",[KEITH A. WADDLE],[19950901],"[Language & Literature, Humanities]",10.2307/43595509,8900103f-bb17-38f8-94aa-ad77ca8ca07f,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,"[MARY GARTH, THE WOLLSTONECRAFTIAN FEMINIST OF...","[Pity, Women, Passion, Humor, Compassion, Hone...",fla,None,1995,1990,179,"[[[234173, 234366], 44], [[281678, 281814], 30...","[44, 30, 27, 52, 26]"
75,[],[],[ELIZABETH WINSTON],[19950901],"[Language & Literature, Humanities]",10.2307/43595523,7acdda65-ea4c-3c1c-aa20-1f0e59773b19,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,None,"[Novels, Serial publications, Adumbration, Gos...",brv,None,1995,1990,0,[],[]


In [8]:
ge_ghls_1990s_df.to_json('../data/ge_ghls_1990s_data.json')

In [9]:
ge_ghls_2010s_df = ge_ghls_df[ge_ghls_df['Decade'] == 2010]
ge_ghls_2010s_df.head()

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
7,[],[],[K. K. Collins],[20151101],"[Language & Literature, Humanities]",10.5325/georelioghlstud.67.2.0234,6a9b9119-bfa6-3a6e-82cf-1dccbaefe55b,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[Review],"[Biography, Masculinity, Pastiche, Textbook re...",brv,[67],2015,2010,0,[],[]
48,[],[],[Claudia Stevens],[20151101],"[Language & Literature, Humanities]",10.5325/georelioghlstud.67.2.0227,6dfa8330-d590-35d4-8e42-5686f97cf016,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[A New Opera: <em>Middlemarch in Spring</em>],"[Opera, Solos, Orchestras, Uncles, Ragpickers,...",fla,[67],2015,2010,0,[],[]
97,"[[490324, 490390], [513891, 514153], [514696, ...","[[8092, 8157], [9348, 9609], [9852, 10238], [1...",[NICOLE M. COONRADT],[20120901],"[Language & Literature, Humanities]",10.2307/42827900,8a9789c8-ad8c-387a-88ef-37aab78b949f,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,[WRITING MARY GARTH: LOCATING MIDDLE GROUND AM...,"[Women, Omniscient narrators, Ugliness, Pity, ...",fla,None,2012,2010,425,"[[[490324, 490390], 13], [[513891, 514153], 52...","[13, 52, 80, 41, 49, 129, 61]"
119,[],[],[MICAEL M. CLARKE],[20131001],"[Language & Literature, Humanities]",10.2307/42827927,4a3436b7-92f0-38bf-82b4-750744974234,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,None,"[Novels, Crime fiction, Literary characters, M...",brv,None,2013,2010,0,[],[]
146,[],[],"[J. HILLIS MILLER, DONALD HAWES, JOHN R. PFEIF...",[20100901],"[Language & Literature, Humanities]",10.2307/42827878,9439fdb7-36c1-36d4-8b45-c480d2b5c4c8,[georelioghlstud],George Eliot - George Henry Lewes Studies,[eng],...,<cite>George Eliot - George Henry Lewes Studie...,None,"[Literary criticism, Humor, Irony, Humanism, S...",brv,None,2010,2010,0,[],[]


In [10]:
ge_ghls_2010s_df.to_json('../data/ge_ghls_2010s_data.json')

## *ELH*

In [11]:
elh_df = df.loc[df['jcode'].str[0] == 'elh']
elh_df

,Locations in A,Locations in B,author,coverdate,disc_name,doi,id,jcode,journal,la,...,srcHtml,title,topics,ty,vo,year,Decade,Quoted Words,Locations in A with Wordcounts,Wordcounts
19,"[[415187, 415972], [1691326, 1691647]]","[[8469, 9251], [18464, 18785]]",[Janet K. Gezari],[19780401],"[Language & Literature, Humanities]",10.2307/2872453,bdec7777-4547-380f-8567-cb4dbcaa9388,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 45, No. 1 (Spring, 1978...",[The Metaphorical Imagination of George Eliot],"[Novels, Sympathy, Travel, Dialectic, Allegory...",fla,[45],1978,1970,228,"[[[415187, 415972], 158], [[1691326, 1691647],...","[158, 70]"
26,"[[698796, 699017], [1368267, 1368353], [169094...","[[15380, 15597], [29093, 29180], [40262, 40962]]",[Brian Swann],[19720601],"[Language & Literature, Humanities]",10.2307/2872247,af76839c-e17f-336c-a45b-7e2a150cf963,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 39, No. 2 (Jun., 1972),...",[Middlemarch: Realism and Symbolic Form],"[Synecdoche, Irony, Pity, Symbolism, Chivalry,...",fla,[39],1972,1970,231,"[[[698796, 699017], 57], [[1368267, 1368353], ...","[57, 15, 159]"
28,"[[18311, 18654], [36712, 36879], [39525, 39677...","[[9299, 9639], [13482, 13644], [22259, 22414],...",[Sophia Andres],[19881201],"[Language & Literature, Humanities]",10.2307/2873139,682a0b66-d9b2-305c-99dd-22cd29b16067,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 55, No. 4 (Winter, 1988...",[The Germ and the Picture in Middlemarch],"[Labyrinths, Preludes, Diastole, Harps, Curtai...",fla,[55],1988,1980,516,"[[[18311, 18654], 79], [[36712, 36879], 36], [...","[79, 36, 26, 46, 20, 15, 17, 29, 34, 15, 68, 1..."
72,"[[291679, 291940], [297602, 297690], [307281, ...","[[1777, 2037], [4485, 4590], [5770, 5987], [61...",[Jeremy Tambling],[19901201],"[Language & Literature, Humanities]",10.2307/2873091,4754a86e-dbdb-3cf2-aa1a-0179494fcc8c,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 57, No. 4 (Winter, 1990...","[Middlemarch, Realism and the Birth of the Cli...","[Metonymy, Constructive empiricism, Positivism...",fla,[57],1990,1990,466,"[[[291679, 291940], 52], [[297602, 297690], 17...","[52, 17, 44, 24, 53, 49, 79, 83, 37, 28]"
104,"[[176340, 176439], [312053, 312429], [739568, ...","[[8798, 8898], [15587, 15991], [18709, 18985],...",[MARK ALLISON],[20111001],"[Humanities, Language & Literature]",10.2307/41236564,16ed41f1-35c6-3da3-86e1-b4ef8e34ece2,[elh],None,[eng],...,"<cite>ELH</cite>, Vol. 78, No. 3 (FALL 2011), ...","[UTOPIAN SOCIALISM, WOMEN'S EMANCIPATION, AND ...","[Utopianism, Womens suffrage movements, Pity, ...",fla,[78],2011,2010,187,"[[[176340, 176439], 22], [[312053, 312429], 80...","[22, 80, 54, 31]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5372,[],[],[John P. Farrell],[19890401],"[Language & Literature, Humanities]",10.2307/2873128,b12be317-31aa-3111-a0ff-27e54d33a1f0,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 56, No. 1 (Spring, 1989...",[Reading the Text of Community in Wuthering He...,"[Dialogism, Narratology, Storytelling, Hierogl...",fla,[56],1989,1980,0,[],[]
5373,[],[],[Amit Yahav-Brown],[20060101],"[Language & Literature, Humanities]",10.2307/30030039,f8de8475-4690-3c3b-ad44-679c28d88ea2,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 73, No. 4 (Winter, 2006...",[Reasonableness and Domestic Fiction],"[Reason, Pluralist school, Letter writing, Par...",fla,[73],2006,2000,0,[],[]
5376,[],[],[William Deresiewicz],[19970701],"[Language & Literature, Humanities]",10.2307/30030146,4321709c-1c11-337b-a7d1-7d45878a15c0,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 64, No. 2 (Summer, 1997...","[Community and Cognition in ""Pride and Prejudi...","[Irony, Humor, Intimacy, Gossip, Flattery, Com...",fla,[64],1997,1990,0,[],[]
5383,[],[],[Rebecca F. Stern],[19980701],"[Language & Literature, Humanities]",10.2307/30030186,0f0a5ad2-54e6-3dc7-8bd7-b8a2c89edbef,[elh],ELH,[eng],...,"<cite>ELH</cite>, Vol. 65, No. 2 (Summer, 1998...",[Moving Parts and Speaking Parts: Situating Vi...,"[Spectacle, S

In [12]:
elh_df.to_json('../data/elh_data.json')

In [13]:
elh_df['Decade'].value_counts()

2000    21
1980    15
1990    14
1970    13
2010    12
1960     6
1950     2
1940     1
1930     1
Name: Decade, dtype: int64

In [14]:
elh_1960s_df = elh_df[elh_df['Decade'] == 1960]
elh_1970s_df = elh_df[elh_df['Decade'] == 1970]
elh_1990s_df = elh_df[elh_df['Decade'] == 1990]
elh_2010s_df = elh_df[elh_df['Decade'] == 2010]

In [15]:
elh_1960s_df.to_json('../data/elh_1960s_data.json')
elh_1970s_df.to_json('../data/elh_1970s_data.json')
elh_1990s_df.to_json('../data/elh_1990s_data.json')
elh_2010s_df.to_json('../data/elh_2010s_data.json')

In [10]:
df_2010s_df = df[df['Decade'] == 2010]
df_2010s_df.to_json('../data/df_2010s_data.json')

In [56]:
df_1980s_and_1990s_df = df[df['Decade'] == (1980 or 1990)]
df_1980s_and_1990s_df.to_json('../data/df_1980s_and_1990s_df_data.json')

## Browsers


In [88]:
## Change the JSON file for different dataframes
name_of_setting = 'victorian_studies'
df = pd.read_json(f'../data/{name_of_setting}.json')

In [7]:
# Adapted from text-matcher
class Text: 
    def __init__(self, filename): 
        self.filename = filename
        
    @property
    def text(self):
        """ Reads the file in memory. """
        f = open(self.filename, encoding='utf-8', errors='ignore')
        return f.read() 

    @property
    def tokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens

In [8]:
mm = Text('../middlemarch.txt')

In [9]:
# Get the size of the text. 
textALength = 1793449

# I don't know why, but without the offset the novel ends too soon,
# with "unvisited tomb." This fixes it. 
offset = 2
textALength += offset

# Make an empty array the size of the text. 
tally = np.zeros(textALength, dtype=np.int64)

In [10]:
len(tally)

1793451

In [11]:
# Read the locations from the CSV file, and literally evaluate them into lists. 
locations = df['Locations in A']
#locations = locations.apply(literal_eval)

In [12]:
#type(locations[0])

In [13]:
# Tally up every time a letter in the novel is quoted. 
for article in locations: 
    for locRange in article: 
         for i in range(locRange[0], min(locRange[1]+1, len(tally))):
                tally[i] += 1

In [14]:
mm.text[1793218:1793418]

'growing good of the world is partly\ndependent on unhistoric acts; and that things are not so ill with you\nand me as they might have been, is half owing to the number who lived\nfaithfully a hidden life'

In [28]:
# Make a color list in hex for all the values in the tally. 
# Let's hope there aren't too many. 
normalized = [int(round((i/tally.max() * 255))) for i in range(tally.max())]
sizeList = [f"{x+14}px" for x in normalized]
#colorList = [rgb2hex(x) for x in cm.gnuplot(normalized)]

In [29]:
sizeList

['14px',
 '19px',
 '23px',
 '28px',
 '33px',
 '37px',
 '42px',
 '46px',
 '51px',
 '56px',
 '60px',
 '65px',
 '70px',
 '74px',
 '79px',
 '84px',
 '88px',
 '93px',
 '97px',
 '102px',
 '107px',
 '111px',
 '116px',
 '121px',
 '125px',
 '130px',
 '135px',
 '139px',
 '144px',
 '148px',
 '153px',
 '158px',
 '162px',
 '167px',
 '172px',
 '176px',
 '181px',
 '186px',
 '190px',
 '195px',
 '199px',
 '204px',
 '209px',
 '213px',
 '218px',
 '223px',
 '227px',
 '232px',
 '237px',
 '241px',
 '246px',
 '250px',
 '255px',
 '260px',
 '264px']

In [ ]:
# Create a CSS Stylesheet for each color value in the map. 
colorCSS = ""
for i, color in zip(range(0, tally.max()+1), colorList): 
    colorCSS += ".c-%s { font-size: %s; }\n" % (i, color)

In [30]:
# Create a CSS Stylesheet for each color value in the map. 
sizeCSS = ""
for i, size in zip(range(0, tally.max()+1), sizeList): 
    sizeCSS += ".c-%s { font-size: %s; }\n" % (i, size)

In [31]:
n = 50

checkpoints = np.linspace(0, textALength, n).round()
checkpoints = [int(point) for point in checkpoints]

In [32]:
def span(val): 
    return '<span class="c-%s">' % val

previousVal = None
for i, valChar in enumerate(zip(tally, mm.text)):
    val, char = valChar[0], valChar[1]
    if previousVal == None: 
        # First character. 
        out = '<span class="c-%s">' % val
    elif val != previousVal: 
        out += '</span><span class="c-%s">' % val
    if i in checkpoints: 
        out += '<a name="b-%s"></a>' % checkpoints.index(i)
    out += char
    previousVal = val

In [33]:
len(df)

5884

In [34]:
# Make a list of valid decades. 
decades = np.arange(1930, 2020, 10)

# Make a dictionary of decades. 
# Values are a list of locations.  
decadeDict = {}
for i, row in df.iterrows():
    decade = row['Decade']
    locations = row['Locations in A']
    if decade not in decadeDict: 
        decadeDict[decade] = locations
    else: 
        decadeDict[decade] += locations 

In [35]:
# Grab the beginnings of quotes. 
decadeStarts = {decade: [item[0] for item in loc] for decade, loc in decadeDict.items()}

decadesBinned = {decade: 
                 np.histogram(locations, bins=n, range=(0, textALength))[0]
                 for decade, locations in decadeStarts.items() if decade in decades}

decadesDF = pd.DataFrame(decadesBinned).T

In [36]:
# Normalize the totals for each section. 
normalizedBlocks = decadesDF.sum() / decadesDF.sum().max()

# Now use the scale that we're already using for the CSS. 
normalizedBlocks = round(normalizedBlocks * tally.max())

In [37]:
blockHTML = '<section id="blocks">'
for i, block in enumerate(normalizedBlocks): 
    blockHTML += '<div class="block b-%s"><a class="block" href="#b-%s">%s</a></div>' % (int(block), i, i)
blockHTML = blockHTML + "</section>"

In [39]:
blockCSS = """
#blocks {  }
.block, .block { 
  width: 30px; 
  height: 30px; 
  display: inline-block;
  color: white; 
  text-align: center;
  text-decoration: none;
  margin-top: 3px; 
}
"""

for i, color in zip(range(0, tally.max()+1), colorList): 
    blockCSS += '.b-%s { background-color: %s; }\n' % (i, color)
sizeCSS += blockCSS

In [40]:
html = """<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet"> 
  <style>
  pre { 
      font-family: Raleway, serif; 
      font-size: 18px; 
    }
  main { 
      width: 40em; 
      margin: 2em auto; 
  }
  %s
  </style>
</head>
<body>%s<main><pre>%s</pre></main></body></html>
""" % (sizeCSS, blockHTML, out)

In [108]:
with open(f'../annotated_{name_of_setting}.html', 'w') as f: 
    f.write(html)
    f.close()

In [41]:
with open(f'../annotated_annotated_by_size_new.html', 'w') as f: 
    f.write(html)
    f.close()